In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# dependencies to run the notebook

# !pip install torch==1.12.1
# !pip install torchmetrics==0.10.2
# !pip install torchvision==0.14.0
# !pip install texttable==1.6.4


<span style="color:darkviolet">
<font size="3">Download the below files from https://drive.google.com/drive/folders/1q50QMurzK9a5l4JBHWjf8VuWcZkbF7PM to run this notebook : <br>
1) train_bert_embeddings.pkl <br>
2) test_bert_embeddings.pkl <br> </font>
</span>


In [3]:
!pip install pickle5

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 256 kB 5.0 MB/s 


In [4]:
import pickle5 as pickle
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from torchvision import datasets, transforms
from torchvision.transforms import ToTensor
from torch.autograd import Variable
import torch.optim as optim
import warnings
import numpy as np
import torch.nn.functional as F
import pandas as pd
from sklearn.metrics import classification_report
from torch.utils.data.sampler import SubsetRandomSampler

warnings.filterwarnings("ignore")


In [5]:
EMBEDDINGS_NAME = "xlnet"


In [6]:
BASE_PATH = "/content/drive/MyDrive/CSCI-544 Group 18/nlp_data"
TRAIN_EMBEDDINGS = f"{BASE_PATH}/embeddings/train_{EMBEDDINGS_NAME}_embeddings.pkl"
TEST_EMBEDDINGS = f"{BASE_PATH}/embeddings/test_{EMBEDDINGS_NAME}_embeddings.pkl"
TRAIN_DATASET_PATH = f"{BASE_PATH}/legal_bert/data/tos_clauses_train.csv"
TEST_DATASET_PATH = f"{BASE_PATH}/legal_bert/data/tos_clauses_dev.csv"
RNN_MODEL_PATH = f"{BASE_PATH}/models/rnn_{EMBEDDINGS_NAME}_model.pt"
GRU_MODEL_PATH = f"{BASE_PATH}/models/gru_{EMBEDDINGS_NAME}_model.pt"
LSTM_MODEL_PATH = f"{BASE_PATH}/models/lstm_{EMBEDDINGS_NAME}_model.pt"
BILSTM_MODEL_PATH = f"{BASE_PATH}/models/bilstm_{EMBEDDINGS_NAME}_model.pt"


In [7]:
# Read train BERT embeddings
with open(TRAIN_EMBEDDINGS, "rb") as f:
    training_data = pickle.load(f)


In [8]:
# Read test BERT embeddings
with open(TEST_EMBEDDINGS, "rb") as f:
    testing_data = pickle.load(f)


In [9]:
# check
for item in training_data:
    print("The data is : ", item)


The data is :  embeddings


In [10]:
len(training_data["embeddings"]), training_data["embeddings"][0].shape


(7531, torch.Size([512, 768]))

<span style="color:darkviolet">
<font size="4">Get the labels from train and test files.</font>
</span>


In [11]:
TRAIN_DATASET_PATH = f"{BASE_PATH}/tos_clauses_train.csv"
TEST_DATASET_PATH = f"{BASE_PATH}/tos_clauses_dev.csv"


In [12]:
train_df = pd.read_csv(TRAIN_DATASET_PATH, header=0)
test_df = pd.read_csv(TEST_DATASET_PATH, header=0)


In [13]:
train_targets = train_df.label.values
test_targets = test_df.label.values


In [14]:
test_targets


array([0, 0, 0, ..., 0, 1, 0])

In [15]:
device = None
if torch.cuda.is_available():
    device = torch.device("cuda")
# elif torch.backends.mps.is_available():
#     device = torch.device("mps")
else:
    device = torch.device("cpu")

print(f"Using Device: {device}")


Using Device: cuda


<span style="color:darkviolet">
<font size="4">Create Dataset, Train and Test Classes</font>
</span>


In [16]:
class Dataset(object):
    """An abstract class representing a Dataset.
    All other datasets should subclass it. All subclasses should
    override ``__len__``, that provides the size of the dataset,
    and ``__getitem__``, supporting integer indexing in range
    from 0 to len(self) exclusive.
    """

    def __getitem__(self, index):
        raise NotImplementedError

    def __len__(self):
        raise NotImplementedError

    def __add__(self, other):
        return ConcatDataset([self, other])


In [17]:
class TOSDataset(Dataset):
    def __init__(self, X, Y, transform=None):
        self.data1 = X
        self.data2 = Y
        self.transform = transform

    def __len__(self):
        return len(self.data1)

    def __getitem__(self, index):
        x = self.data1[index]
        y = self.data2[index]

        if self.transform is not None:
            x = torch.tensor(x)

        return torch.squeeze(x, dim=1), torch.tensor(y)


In [18]:
test_len = len(test_df)
train_len = len(train_df)
X_train_tensor = TOSDataset(train_df["sentences"], train_df["label"])
# X_test_tensor = Train_Model(test_df)

num_train = len(X_train_tensor)
indices = list(range(num_train))
np.random.shuffle(indices)
# split = int(np.floor(num_train))
# train_idx = indices[split:]

train_sampler = SubsetRandomSampler(indices)
# valid_sampler = SubsetRandomSampler(valid_idx)
print(train_sampler)
train_df_by_index = train_df.loc[indices]
# val_df_by_index = df_train.loc[valid_idx]
train_fair = sum(train_df_by_index["label"] == 0)
train_unfair = sum(train_df_by_index["label"] == 1)
# val_fair = sum(val_df_by_index['label'] == 0)
# val_unfair = sum(val_df_by_index['label'] == 1)

print("train_fair:" + str(train_fair))
print("train_unfair:" + str(train_unfair))
# print("val_fair:" + str(val_fair))
# print("val_unfair:" + str(val_unfair))


train_fair:6705
train_unfair:826


In [19]:
train_data = TOSDataset(training_data["embeddings"], train_targets, transform=transforms.ToTensor())
test_data = TOSDataset(testing_data["embeddings"], test_targets, transform=transforms.ToTensor())


<span style="color:darkviolet">
<font size="4">Prepare Data loaders</font>
</span>


In [20]:
# how many samples per batch to load
BATCH_SIZE = 20

# number of subprocesses to use for data loading
NUM_WORKERS = 0


In [21]:
# prepare data loaders
train_loader = DataLoader(train_data, batch_size=BATCH_SIZE, sampler=train_sampler, num_workers=NUM_WORKERS)
test_loader = DataLoader(test_data, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS)


In [22]:
# check sizes
dataiter = iter(train_loader)
sample_x, sample_y = dataiter.next()

print("Sample input size: ", sample_x.size())  # batch_size, seq_length
# print("Sample input: \n", sample_x)
print()
print("Sample label size: ", sample_y.size())  # batch_size
# print("Sample label: \n", sample_y)


Sample input size:  torch.Size([20, 512, 768])

Sample label size:  torch.Size([20])


In [23]:
torch.squeeze(sample_x, dim=1).shape


torch.Size([20, 512, 768])

In [45]:
# LSTM

class LSTM_Network(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        
        super(LSTM_Network, self).__init__()

         # Number of hidden dimensions
        self.hidden_dim = hidden_dim
        
        # RNN
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers=1, batch_first=True)
        
        # Readout layer
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        
        # Initialize hidden state with zeros
        h0 = Variable(torch.zeros(1, x.size(0), self.hidden_dim)).to(device)
        c0 = Variable(torch.zeros(1, x.size(0), self.hidden_dim)).to(device)
        
        # One time step
        out, (hn, cn) = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        
        return out

In [46]:
import time


def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs


In [47]:
EMBEDDING_DIM = 768
OUTPUT_DIM = 2


In [49]:
HIDDEN_DIM = 512
N_EPOCHS = 30


In [50]:
from sklearn.utils.class_weight import compute_class_weight

class_weight = compute_class_weight(
    "balanced", classes=np.unique(train_df_by_index["label"]), y=train_df_by_index["label"]
)
class_weight


array([0.56159582, 4.55871671])

In [51]:
model = LSTM_Network(EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM).to(device)
model = model.to(device)

loss_fn = nn.CrossEntropyLoss(weight=torch.FloatTensor(class_weight)).to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-3)
test_min_loss = np.inf

for epoch in range(N_EPOCHS):

    start_time = time.time()
    model.train()
    train_loss = 0.0
    test_loss = 0.0
    for inputs, target in train_loader:
        inputs, target = inputs.to(device), target.to(device)
        optimizer.zero_grad()
        inputs = torch.squeeze(inputs, dim=1)
        output = model(inputs)
        loss = loss_fn(output, target)
        loss.backward()
        # do gradient clipping
        # torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=4.0, norm_type=2)
        optimizer.step()
        train_loss += loss.item()

    model.eval()
    for inputs, target in test_loader:
        inputs, target = inputs.to(device), target.to(device)
        inputs = torch.squeeze(inputs, dim=1)
        output = model(inputs)
        loss = loss_fn(output, target)
        test_loss += loss.item()

    train_loss = train_loss / len(train_loader.dataset)
    test_loss = test_loss / len(test_loader.dataset)

    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    print(f"Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s")
    print("\tTraining Loss: {:.6f} \Test Loss: {:.6f}".format(train_loss, test_loss))
    if test_loss <= test_min_loss:
        print("Test loss decreased ({:.6f} --> {:.6f}). Saving model...".format(test_min_loss, test_loss))
        torch.save(model.state_dict(), LSTM_MODEL_PATH)
        test_min_loss = test_loss


Epoch: 01 | Epoch Time: 0m 16s
	Training Loss: 0.031473 \Test Loss: 0.025899
Test loss decreased (inf --> 0.025899). Saving model...
Epoch: 02 | Epoch Time: 0m 16s
	Training Loss: 0.024489 \Test Loss: 0.023378
Test loss decreased (0.025899 --> 0.023378). Saving model...
Epoch: 03 | Epoch Time: 0m 16s
	Training Loss: 0.020638 \Test Loss: 0.021605
Test loss decreased (0.023378 --> 0.021605). Saving model...
Epoch: 04 | Epoch Time: 0m 16s
	Training Loss: 0.018020 \Test Loss: 0.018290
Test loss decreased (0.021605 --> 0.018290). Saving model...
Epoch: 05 | Epoch Time: 0m 16s
	Training Loss: 0.015889 \Test Loss: 0.016584
Test loss decreased (0.018290 --> 0.016584). Saving model...
Epoch: 06 | Epoch Time: 0m 16s
	Training Loss: 0.014476 \Test Loss: 0.016632
Epoch: 07 | Epoch Time: 0m 16s
	Training Loss: 0.013315 \Test Loss: 0.015552
Test loss decreased (0.016584 --> 0.015552). Saving model...
Epoch: 08 | Epoch Time: 0m 16s
	Training Loss: 0.011981 \Test Loss: 0.014766
Test loss decreased (0.

In [53]:
y_pred_list = []
y_targ_list = []
model = LSTM_Network(EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM).to(device)
model.load_state_dict(torch.load(LSTM_MODEL_PATH))
model.eval()

with torch.no_grad():
    for inputs, target in test_loader:
        inputs, target = inputs.to(device), target.to(device)
        inputs = torch.squeeze(inputs, dim=1)
        y_test_pred = model(inputs)
        _, y_test_pred = torch.max(y_test_pred, 1)
        y_pred_tag = y_test_pred
        y_pred_list.append(y_pred_tag.cpu().numpy())
        y_targ_list.append(target.cpu().numpy())

y_pred_list = [x.squeeze().tolist() for x in y_pred_list]
y_targ_list = [x.squeeze().tolist() for x in y_targ_list]
y_pred_list = [x for sublist in y_pred_list for x in sublist]
y_targ_list = [x for sublist in y_targ_list for x in sublist]

print(classification_report(y_targ_list, y_pred_list))


              precision    recall  f1-score   support

           0       0.98      0.90      0.94      1677
           1       0.51      0.85      0.64       206

    accuracy                           0.90      1883
   macro avg       0.75      0.88      0.79      1883
weighted avg       0.93      0.90      0.91      1883

